# stdpopsim 测试和调试

用于测试stdpopsim API的兼容性

In [ ]:
import stdpopsim
import tskit
import numpy as np
import matplotlib.pyplot as plt

print(f"stdpopsim version: {stdpopsim.__version__}")
print(f"tskit version: {tskit.__version__}")

In [ ]:
# 测试基本API
species = stdpopsim.get_species("HomSap")
print(f"Species: {species.name}")
print(f"Generation time: {species.generation_time} years")
print(f"Common name: {species.common_name}")

In [ ]:
# 测试人口模型
try:
    model = species.get_demographic_model("OutOfAfrica_3G09")
    print(f"Demographic model: {model.id}")
    print(f"Populations: {[pop.name for pop in model.populations]}")
except Exception as e:
    print(f"Error getting demographic model: {e}")
    # 尝试列出可用的模型
    print("Available models:")
    for model in species.demographic_models:
        print(f"  - {model.id}: {model.description}")

In [ ]:
# 测试contig配置
try:
    # 使用新的API方式，指定具体的长度范围
    contig = species.get_contig("chr22", left=0, right=1_000_000)  # 使用1Mb进行快速测试
    print(f"Chromosome: chr22")
    print(f"Length: {contig.length:,} bp")
    print(f"Mutation rate: {contig.mutation_rate:.2e}")
    if hasattr(contig, 'recombination_map'):
        print(f"Recombination rate: {contig.recombination_map.mean_rate:.2e}")
except Exception as e:
    print(f"Error getting contig: {e}")
    # 尝试列出可用的染色体
    print("Available chromosomes:")
    for chrom in species.genome.chromosomes:
        print(f"  - {chrom.id}: {chrom.length:,} bp")

In [ ]:
# 测试简单模拟
try:
    # 使用最简单的配置
    samples = {"CHB": 10}  # 只用10个样本进行快速测试
    
    engine = stdpopsim.get_engine("msprime")
    ts = engine.simulate(
        demographic_model=model,
        contig=contig,
        samples=samples,
        seed=42
    )
    
    print(f"\n模拟成功!")
    print(f"样本数: {ts.num_samples}")
    print(f"树数量: {ts.num_trees:,}")
    print(f"突变数: {ts.num_mutations:,}")
    print(f"序列长度: {ts.sequence_length:,.0f} bp")
    
    # 基本统计
    diversity = ts.diversity()
    print(f"核苷酸多样性 (π): {diversity:.6f}")
    
except Exception as e:
    print(f"模拟失败: {e}")
    import traceback
    traceback.print_exc()